In [1]:
%matplotlib inline
%load_ext tensorboard

import os
import tempfile

import torch
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger

from torchgeo.datamodules import EuroSAT100DataModule
from torchgeo.models import ResNet18_Weights
from torchgeo.trainers import ClassificationTask

In [2]:
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss", dirpath="logs/RoofSense", save_top_k=1, save_last=True
)
early_stopping_callback = EarlyStopping(monitor="val_loss", min_delta=0.00, patience=10)
logger = TensorBoardLogger(save_dir="logs/RoofSense", name="tutorial_logs")

In [3]:
from lightning import Trainer
from torchgeo.trainers import SemanticSegmentationTask

from classification.datamodules import TrainingDataModule

datamodule = TrainingDataModule(  # Dataset Options
    paths="../pretraining",  # Data Module Options
    batch_size=16,
    patch_size=64,
    length=4096,
    num_workers=10,
)
task = SemanticSegmentationTask(
    model="unet",
    backbone="resnet18",
    weights=False,
    in_channels=6,
    num_classes=9,
    loss="jaccard",
    ignore_index=None,
    lr=0.00001,
    patience=6,
)

In [7]:
%tensorboard --logdir "logs/RoofSense" --host localhost --port 8090

In [8]:
trainer = Trainer(
    callbacks=[checkpoint_callback, early_stopping_callback],
    log_every_n_steps=1,
    logger=logger,
    max_epochs=10
)

trainer.fit(model=task, datamodule=datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
C:\Documents\RoofSense\venv\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:639: Checkpoint directory C:\Documents\RoofSense\classification\logs\RoofSense exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | JaccardLoss      | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0  

The epoch length is4096


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

C:\Documents\RoofSense\venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
C:\Documents\RoofSense\venv\Lib\site-packages\matplotlib\cm.py:494: RuntimeWarning: invalid value encountered in cast
  xx = (xx * 255).astype(np.uint8)
C:\Documents\RoofSense\venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Training: |          | 0/? [00:00<?, ?it/s]

C:\Documents\RoofSense\venv\Lib\site-packages\torch\nn\functional.py:4358: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
C:\Documents\RoofSense\venv\Lib\site-packages\torch\nn\functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

C:\Documents\RoofSense\venv\Lib\site-packages\lightning\pytorch\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [12]:
trainer.test(model=task,datamodule=datamodule)

In [13]:
model = task.model.to("cpu").eval()
model

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [14]:
from tqdm import tqdm
import numpy as np
y_preds = []
y_trues = []
for batch in tqdm(datamodule.test_dataloader()):
    images = batch["image"].to("cpu")
    y_trues.append(batch["mask"].numpy().ravel()[::500])
    with torch.inference_mode():
        y_pred = model(images).argmax(dim=1).cpu().numpy().ravel()[::500]
    y_preds.append(y_pred)

y_preds = np.concatenate(y_preds)
y_trues = np.concatenate(y_trues)


KeyboardInterrupt



In [ ]:
for batch in datamodule.val_dataloader():
    print(batch["bbox"])

In [ ]:


minibatch_id=5
img=images[minibatch_id,:,:,:]
msk=batch["mask"].to("cpu")[minibatch_id,:,:,:].squeeze()
prd=model(images).argmax(dim=1).cpu()[minibatch_id,:,:]



In [ ]:
import matplotlib.pyplot as plt
plt.imshow(msk)

In [ ]:
plt.imshow(prd)

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(y_trues, y_preds,average="micro")
recall = recall_score(y_trues, y_preds,average="micro")
f1 = 2 * (precision * recall) / (precision + recall)
print(precision,recall,f1)